# SWOT Feature Translation Service (FTS) Examples

This is the associated Jupyter Notebook containing four examples related to querying the SWOT Feature Translation Service and using its results to query NASA's Common Metadata Repository (CMR).

In [ ]:
%%capture

import sys
!{sys.executable} -m pip install bs4 requests

In [ ]:
from bs4 import BeautifulSoup
import requests
import json

## Exact SWOT Feature ID Matches in the SWOT Feature Translation Service

Below you can input some given SWOT Feature ID and query the Feature Translation Service based on it.

In [ ]:
###################

# Mimicing a user querying exact matches with SWOT Feature ID "7541140001000000"

SWOT_FEATURE_ID = "7541140001000000"
EXACT = True

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that exactly match SWOT Feature ID "75411400010000"
print(json.dumps(response, indent = 4))

## Partial SWOT Feature ID Matches in the SWOT Feature Translation Service

In a similar fashion to the HUC FTS portion, you can query the service for partial matches to the SWOT Feature ID. This will return a string formatted to directly be used in a CMR query.

In [ ]:
###################

# Mimicing a user querying partial matches with SWOT Feature ID "75411400010000"

SWOT_FEATURE_ID = "75411400010000"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)

# Print all elements in HUC database that exactly match HUC "180500030105"
print(json.dumps(response, indent = 4))

## Query CMR by String

Below is an example of how the output of the SWOT Feature Translation Service portion can directly be used to query CMR. I'm using the given SWOT Feature ID to search for granules available through the Sentinel-1 mission. Currently this is through a string, however this isn't technically correct as we're _AND'ing_ spatial features together rather than _OR'ing_ them. 

As of now, CMR does not support POSTing a JSON body of spatial parameters, however this is an area actively being looked into.

In [ ]:
###################

# Mimicing a user querying exact matches with SWOT Feature ID "75411400010000"

COLLECTION_ID = "C1522341104-NSIDC_ECS" # SMAP/Sentinel-1 L2 Radiometer/Radar 30-Second Scene 3 km EASE-Grid Soil Moisture V002
SWOT_FEATURE_ID = "75411400010000"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)

geo_list = response['results'][SWOT_FEATURE_ID]
#print(geo_list)

# Query CMR
# --------- #

#cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules.json?{}&echo_collection_id=C1522341104-NSIDC_ECS&pretty=True".format(polygon))
cmr_response = requests.get("https://cmr.earthdata.nasa.gov/search/granules?{}&echo_collection_id={}&pretty=True".format(geo_list, COLLECTION_ID))

# --------- #

# Make it look nice
soup = BeautifulSoup(cmr_response.text, features = 'lxml')
print(soup.prettify())

## Query CMR via JSON

As mentioned in the previous example, this is not currently possible, however I've added direct support for when it is. Once available, a proverbial lever can be flicked to allow querying via JSON. The code below is something that can be used once possible.

In [ ]:
###################

# Mimicing a user querying exact matches with SWOT Feature ID "75411400010000"

COLLECTION_ID = "C1522341104-NSIDC_ECS" # SMAP/Sentinel-1 L2 Radiometer/Radar 30-Second Scene 3 km EASE-Grid Soil Moisture V002
SWOT_FEATURE_ID = "75411400010000"
EXACT = False

###################

# Query Feature Translation Service and parse JSON response
r = requests.get("https://g6zl7z2x7j.execute-api.us-west-2.amazonaws.com/prod/swot/{}?exact={}".format(SWOT_FEATURE_ID, EXACT))

# Load response from FTS
response = json.loads(r.text)
json_response = response['results'][SWOT_FEATURE_ID]
#print(json_response)

# Query CMR
# --------- #

#cmr_response = requests.post("https://cmr.earthdata.nasa.gov/search/granules?echo_collection_id={}&pretty=True".format(COLLECTION_ID), data = json_response)
cmr_response = requests.post("https://cmr.earthdata.nasa.gov/search/granules?echo_collection_id={}&pretty=True".format(COLLECTION_ID), data = json_response)

# --------- #

# Make it look nice
soup = BeautifulSoup(cmr_response.text, features = 'lxml')
print(soup.prettify())